Importing Libraries

In [2]:
import cv2
import time 

import tensorflow as tf
import os
import shutil
import numpy as np
import random
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from pathlib import Path
import re
import cv2
from keras import backend as K
tf.__version__
import glob

#check if tensorflow gpu is being used
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("tensorflow version:", tf.__version__)

Num GPUs Available:  1
tensorflow version: 2.10.0


In [3]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

ROI designation

In [12]:
#Background Subtraction
#reference: https://www.geeksforgeeks.org/python-background-subtraction-using-opencv/?ref=lbp, 
#Image blurring reference: https://www.geeksforgeeks.org/python-image-blurring-using-opencv/ 


#videos to choose from
from numpy import zeros_like


NeedleViz_path1 = 'Data/edited data/102622_Water.mp4'
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4'

#control playback speed
frame_rate = 25

# vc = cv2.VideoCapture(0) #opens camera
vc = cv2.VideoCapture(NeedleViz_path1)

prev_frame_time = 0
new_frame_time = 0
average_fps_list = []

frameWidth = 440
frameHeight = 440
vc.set(3, frameWidth)
vc.set(4, frameHeight)


def empty(a):
  pass


cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters",640,240)
cv2.createTrackbar("col_initial","Parameters",100,440,empty)
cv2.createTrackbar("col_final","Parameters",300,440,empty)
cv2.createTrackbar("row_initial","Parameters",100,440,empty)
cv2.createTrackbar("row_final","Parameters",200,440,empty)


if (vc.isOpened()== False): 
  print("Error opening video  file")

while(vc.isOpened()):
    rval, frame = vc.read()
    
    if rval == True:
        #Insert Code Here
        col_initial = cv2.getTrackbarPos("col_initial", "Parameters")
        col_final = cv2.getTrackbarPos("col_final", "Parameters")
        row_initial = cv2.getTrackbarPos("row_initial", "Parameters")
        row_final = cv2.getTrackbarPos("row_final", "Parameters")

        resized_frame = cv2.resize(frame, (frameWidth,frameHeight))
        resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_RGB2GRAY)
        ROI_frame = resized_frame[94:348, 166:275]
         # ROI_frame = resized_frame[col_initial:col_final,row_initial:row_final]
        blank_img = zeros_like(resized_frame)
        
        x = 94 #initial column number
        y = 166 #initial row number

        for i in range(0, 254):
          for j in range(0, 109):

            if ROI_frame[i][j] != 0:
              blank_img[x + i, y + j] = ROI_frame[i, j] 
       
       
        #Debugging statements
        # print(ROI_frame.shape)
        # print(ROI_frame[0][1])
        # print(blank_img[0][0])


        cv2.imshow('normal frame', resized_frame)
        cv2.imshow('bounded region', ROI_frame)
        cv2.imshow('testing', blank_img)

        # Press Q on keyboard to  exit
        if cv2.waitKey(frame_rate) & 0xFF == ord('q'): #original waitkey is 25
            break
    
    #Break out of loop if video is done
    else:
        break  

vc.release() #Release the video capture object

# Close window
cv2.destroyAllWindows()

Combintations of Background Subtraction + Denoising + ROI

In [28]:
#Background Subtraction
#reference: https://www.geeksforgeeks.org/python-background-subtraction-using-opencv/?ref=lbp, 
#Image blurring reference: https://www.geeksforgeeks.org/python-image-blurring-using-opencv/ 


#videos to choose from
NeedleViz_path1 = 'Data/edited data/102622_Water.mp4'
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4'

#control playback speed
frame_rate = 25

# vc = cv2.VideoCapture(0) #opens camera
vc = cv2.VideoCapture(NeedleViz_path2)

fgbg = cv2.createBackgroundSubtractorMOG2(20,50)

prev_frame_time = 0
new_frame_time = 0
average_fps_list = []

frameWidth = 440
frameHeight = 440
vc.set(3, frameWidth)
vc.set(4, frameHeight)

# cv2.namedWindow("Parameters")
# cv2.resizeWindow("Parameters",640,240)
# cv2.createTrackbar("alpha","Parameters", 255, 255, empty)
# cv2.createTrackbar("beta","Parameters", 197, 255, empty)

def empty(a):
  pass

if (vc.isOpened()== False): 
  print("Error opening video  file")

while(vc.isOpened()):
    rval, frame = vc.read()
    
    if rval == True:
        #Insert Code Here

        resized_frame = cv2.resize(frame, (frameWidth,frameHeight))
        ROI_frame = resized_frame[94:348, 166:275]

        fgmask = fgbg.apply(ROI_frame)

        #Applying Gaussian Filter
        imgBlur = cv2.GaussianBlur(ROI_frame,(7,7), 1)
        contrast_imgBlur = 255*imgBlur+100

        fgmaskV2 = fgbg.apply(contrast_imgBlur)
             
        cv2.imshow('normal frame', resized_frame)
        cv2.imshow('FGmask', fgmask)
        cv2.imshow('FGmaskV2', fgmaskV2)

        # Press Q on keyboard to  exit
        if cv2.waitKey(frame_rate) & 0xFF == ord('q'): #original waitkey is 25
            break
    
    #Break out of loop if video is done
    else:
        break  

vc.release() #Release the video capture object

# Close window
# cv2.destroyWindow("normal frame")
# cv2.destroyWindow("FGmask")
cv2.destroyAllWindows()

Overlay on B-mode 

In [26]:
#videos to choose from
NeedleViz_path1 = 'Data/edited data/102622_Water.mp4'
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4'

#control playback speed
frame_rate = 25

# vc = cv2.VideoCapture(0) #opens camera
vc = cv2.VideoCapture(NeedleViz_path2)

fgbg = cv2.createBackgroundSubtractorMOG2()

prev_frame_time = 0
new_frame_time = 0
average_fps_list = []

frameWidth = 440
frameHeight = 440
vc.set(3, frameWidth)
vc.set(4, frameHeight)

if (vc.isOpened()== False): 
  print("Error opening video  file")

while(vc.isOpened()):
    rval, frame = vc.read()
    
    if rval == True:
        #Insert Code Here

        resized_frame = cv2.resize(frame, (frameWidth,frameHeight))
        resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_RGB2GRAY)
        ROI_frame = resized_frame[94:348, 166:275]
         # ROI_frame = resized_frame[col_initial:col_final,row_initial:row_final]
        blank_img = zeros_like(resized_frame)

        x = 94 #initial column number
        y = 166 #initial row number

        for i in range(0, 254):
          for j in range(0, 109):

            if ROI_frame[i][j] != 0:
              blank_img[x + i, y + j] = ROI_frame[i, j] 

        fgmask = fgbg.apply(blank_img)

        #Applying Gaussian Filter
        imgBlur = cv2.GaussianBlur(blank_img,(7,7), 1)
        contrast_imgBlur = 255*imgBlur+100

        fgmaskV2 = fgbg.apply(contrast_imgBlur)

        fgmaskV2_color = cv2.applyColorMap(fgmaskV2, cv2.COLORMAP_MAGMA)

        resized_frame_revert = cv2.cvtColor(resized_frame, cv2.COLOR_GRAY2RGB)

        # Debugging Statements
        # cv2.imshow('normal frame', resized_frame)
        # cv2.imshow('FGmask', fgmask)
        # cv2.imshow('FGmaskV2', fgmaskV2_color)
        # print("raw image shape = {}".format(resized_frame.shape))
        # print("segmented image shape = {}".format(fgmaskV2_color.shape))
        # print("raw image shape (reverted)= {}".format(resized_frame_revert.shape))


        overlay = cv2.addWeighted(resized_frame_revert, 0.5, fgmaskV2_color, 0.5, 1.0)
        cv2.imshow("Bmode Overlay", overlay)

        # Press Q on keyboard to  exit
        if cv2.waitKey(frame_rate) & 0xFF == ord('q'): #original waitkey is 25
            break
    
    #Break out of loop if video is done
    else:
        break  

vc.release() #Release the video capture object

# Close window
# cv2.destroyWindow("normal frame")
# cv2.destroyWindow("FGmask")
cv2.destroyAllWindows()